# Data Organization Script # 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import pi
# import seaborn as sns
# import matplotlib as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


Data
Data_Organization.ipynb
GiantKillerInfo.txt
logs
MarchMadnessAdvStats.csv
MarchMadnessFeatureDifferences.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
ModelEvaluation.ipynb
OldNbs
SubmissionData.py
Submissions
tENSORFLOW.ipynb
tENSORFLOW.py
tENSORFLOW-TB.ipynb



### Lock Random Seed

In [2]:
seed = 13
np.random.seed(seed)

# Load and Organize Training data


In [3]:
data_dir = '../NCAA/Data/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


Incorporate Massey Ordinals - MAS, SAG, POM

In [4]:
#Load Data
df_rank = pd.read_csv(data_dir+ 'MasseyOrdinals.csv')
#Choose Latest Ranking 
df_rank = df_rank[df_rank.RankingDayNum>=133]
df_rank = df_rank[df_rank.Season>=2008]

#Selectr rankings of interest and make each ranking system ranking into a separate column
df_merge = pd.merge(df_rank.loc[df_rank['SystemName']=='MAS'],
                    df_rank.loc[df_rank['SystemName']=='SAG'], how='left', 
                    on=['Season', 'TeamID', 'RankingDayNum'])
df_rank = pd.merge(left=df_merge, right=df_rank.loc[df_rank['SystemName']=='POM'], 
                  how='left', on=['Season', 'TeamID', 'RankingDayNum'] )


Remove unnecessary cols

In [5]:
# Drop
df_rank.drop(labels=['SystemName_x', 'SystemName_y', 'SystemName','RankingDayNum'], inplace=True, axis =1)
df_rank.rename(columns={'OrdinalRank_x':'MAS', 'OrdinalRank_y':'SAG', 'OrdinalRank':'POM'}, inplace=True)

# df_rank.head()

Add advanced stats from https://www.kaggle.com/lnatml/feature-engineering-with-advanced-stats/notebook

In [6]:
df = pd.read_csv(data_dir+'RegularSeasonDetailedResults.csv')


In [7]:
#Points Winning/Losing Team
df['WPts'] = df.apply(lambda row: 2*(row.WFGM-row.WFGM3) + 3*row.WFGM3 + row.WFTM, axis=1)
df['LPts'] = df.apply(lambda row: 2*(row.LFGM-row.WFGM3) + 3*row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = df.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
df['Pos'] = (wPos+lPos)/2

#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
df['WOffRtg'] = df.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
df['LOffRtg'] = df.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
df['WDefRtg'] = df.LOffRtg
df['LDefRtg'] = df.WOffRtg
#Net Rating = Off.eff - Def.eff
df['WNetRtg'] = df.apply(lambda row:(row.WOffRtg - row.LDefRtg), axis=1)
df['LNetRtg'] = df.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
df['WAstR'] = df.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df['LAstR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO
df['WTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
df['LTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
df['WTSP'] = df.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
df['LTSP'] = df.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
df['WeFGP'] = df.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df['LeFGP'] = df.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
df['WFTAR'] = df.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df['LFTAR'] = df.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
df['WORP'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df['LORP'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
df['WDRP'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df['LDRP'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
df['WRP'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
df['LRP'] = df.apply(lambda row: (row.LDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)

After creating adv stats, now we need to split the winning and losing team stats for a more overall picture

In [8]:
df_TeamID = pd.concat((df['WTeamID'], df['LTeamID']), axis=1)
df_adv2 = df.iloc[:, 34:].copy()
df_adv = pd.concat((df_TeamID, df_adv2), axis=1)
df_adv = pd.concat((df['Season'], df_adv), axis=1)
# df_adv.head()
names = df_adv.columns.values 
print(names)


['Season' 'WTeamID' 'LTeamID' 'WPts' 'LPts' 'Pos' 'WOffRtg' 'LOffRtg'
 'WDefRtg' 'LDefRtg' 'WNetRtg' 'LNetRtg' 'WAstR' 'LAstR' 'WTOR' 'LTOR'
 'WTSP' 'LTSP' 'WeFGP' 'LeFGP' 'WFTAR' 'LFTAR' 'WORP' 'LORP' 'WDRP' 'LDRP'
 'WRP' 'LRP']


In [9]:
Wnames = ['Season', 'WTeamID','WPts','Pos', 'WOffRtg' ,'WDefRtg',
          'WNetRtg', 'WAstR', 'WTOR', 'WTSP','WeFGP','WFTAR', 'WORP', 'WDRP','WRP']
Lnames = ['Season', 'LTeamID', 'LPts', 'Pos', 'LOffRtg','LDefRtg',
          'LNetRtg', 'LAstR', 'LTOR','LTSP', 'LeFGP','LFTAR', 'LORP', 'LDRP', 'LRP' ]
df_advW = df_adv.loc[:,Wnames].copy()
df_advL = df_adv.loc[:,Lnames].copy()
df_advW.rename(columns={'WTeamID':'TeamID'}, inplace=True)
df_advL.rename(columns={'LTeamID':'TeamID'}, inplace=True)

Must concat then group the advanced stats to get season averages for each team

In [10]:
names = ['Season', 'TeamID', 'Pts', 'Pos', 'OffRtg','DefRtg',
          'NetRtg', 'AstR', 'TOR','TSP', 'eFGP','FTAR', 'ORP', 'DRP', 'RP' ]
df_advL.columns = names
df_advW.columns = names
df_A = pd.concat((df_advL, df_advW), axis=0, ignore_index=True)
groupedA = df_A.groupby(['Season', 'TeamID'], as_index=False)
df_A = groupedA.agg(np.mean)
# df_A.shape
df_advL = df_A.copy()
df_advW = df_A.copy()


In [11]:
df_Adv= pd.merge(left=df_A, right=df_rank, on=['Season', 'TeamID'])
df_Adv.tail()

,Season,TeamID,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,eFGP,FTAR,ORP,DRP,RP,MAS,SAG,POM
2783,2017,1460,74.300000,68.305280,108.875974,103.867603,-8.236970,15.165959,12.723307,56.206307,0.524174,0.391122,0.295559,0.737647,0.498543,152,162,166
2784,2017,1461,78.933333,72.211840,109.552337,101.226318,-3.493423,14.960753,12.292200,56.646622,0.507965,0.389409,0.221667,0.727887,0.492821,167,141,146
2785,2017,1462,73.735294,65.164800,113.125625,109.118186,-7.885345,15.600786,14.319593,55.147677,0.517891,0.418095,0.348826,0.758134,0.522060,42,32,40
2786,2017,1463,71.178571,65.226514,108.207219,107.436823,-10.709853,16.074839,13.698384,54.493116,0.522992,0.344515,0.311873,0.727941,0.510952,126,134,151
2787,2017,1464,74.187500,71.694600,103.361054,115.307775,-14.868296,14.674567,13.705051,50.675226,0.491127,0.264800,0.261204,0.686746,0.471342,244,287,283


Cut seasons prior to Max Ranking date

In [12]:
df_seeds = df_seeds[df_seeds.Season>=min(df_rank.Season) ]
df_seeds = df_seeds[ df_seeds.Season<=max(df_rank.Season)]
df_tour = df_tour[df_tour.Season>=min(df_rank.Season)]
df_tour = df_tour[ df_tour.Season<=max(df_rank.Season)]
# df_tour.head()

Cut off the region identifier from the seed number

In [13]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
# df_tour.head()

Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [14]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})


df_dummy = pd.merge(left=df_tour, right=df_lossSeeds, how='left', on=['Season', 'LTeamID'])

df_concat = pd.merge(left=df_dummy, right=df_winSeeds, how='left' ,on=['Season', 'WTeamID'])
# df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
# df_concat.head()

 Make a winner and loser DF with seed, relative seed, and win/loss. 

In [15]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
# df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins = pd.merge(left=df_wins, right=df_advW, how='left', on=['Season', 'TeamID'])
df_wins = pd.merge(left=df_wins, right=df_rank, how='left', on=['Season', 'TeamID'])
# df_wins.tail()

In [16]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
# df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses = pd.merge(left=df_losses, right=df_advL, how='left', on=['Season', 'TeamID'])
df_losses = pd.merge(left=df_losses, right=df_rank, how='left',on=['Season', 'TeamID'])
# df_losses.tail()

Prefix the cols with 'Opp' in dummy frames to allow for horizontal concat in order to expand data to be

| winr_stat    losr_stat |
                     
| losr_stat    winr_stat |

In [17]:
df_lossesOpp = df_losses.copy()

df_lossesOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_lossesOpp.columns.values]
df_lossesOpp.rename(columns = dict(new_names), inplace=True)
# df_lossesOpp.tail()

In [18]:
df_winsOpp = df_wins.copy()

df_winsOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_winsOpp.columns.values]
df_winsOpp.rename(columns = dict(new_names), inplace=True)
# df_winsOpp.head()

In [19]:
df_winloss = pd.concat([df_wins, df_lossesOpp], axis=1)
df_losswin = pd.concat([df_losses, df_winsOpp], axis=1)

Divide train and test case

In [20]:

df_winlossTest = df_winloss[df_winloss.Season>2014]
df_losswinTest = df_losswin[df_losswin.Season>2014]

In [21]:
df_winloss = df_winloss[df_winloss.Season<2014]
df_losswin = df_losswin[df_losswin.Season<2014]

# Combine into final dataframe

In [22]:
df_finalData = pd.concat((df_winloss, df_losswin))
results = df_finalData['Result']
df_finalData.drop(labels=['Result'], inplace=True, axis=1)
df_finalData.insert(0, 'Result', results)
df_finalData.shape

(530, 38)

In [23]:
df_diffWin =  df_wins.loc[:,'Pts':'POM'].copy()-df_losses.loc[:,'Pts':'POM'].copy()
df_diffWin = pd.concat((df_wins.loc[:,'Seed']-(df_losses.loc[:,'Seed']), df_diffWin), axis=1)
new_names = [(i,i+'Diff') for i in df_diffWin.columns.values]
df_diffWin.rename(columns = dict(new_names), inplace=True)
df_diffLoss = -df_diffWin.copy()
df_diffWin = pd.concat((df_wins.loc[:,'Result'], df_diffWin), axis=1)
df_diffLoss = pd.concat((df_losses.loc[:,'Result'], df_diffLoss), axis=1)
 

In [24]:
   
df_diffData = pd.concat((df_diffWin, df_diffLoss))
df_diffWin.tail()

,Result,SeedDiff,PtsDiff,PosDiff,OffRtgDiff,DefRtgDiff,NetRtgDiff,AstRDiff,TORDiff,TSPDiff,eFGPDiff,FTARDiff,ORPDiff,DRPDiff,RPDiff,MASDiff,SAGDiff,POMDiff
528,1,-1,-2.704100,-2.474267,0.169129,6.263033,-3.408628,2.340995,1.542610,-2.466236,-0.009512,-0.099510,0.074700,0.023729,0.033788,6,-2,-1
529,1,3,-6.759073,-0.284497,-8.823666,-0.243991,-1.950864,-0.264810,-0.190002,-4.362310,-0.046102,-0.005965,0.021077,-0.021970,-0.009347,29,21,22
530,1,-6,13.502444,0.198006,19.259922,-5.199333,4.576883,3.570702,0.783395,10.349764,0.110787,-0.008690,-0.049340,0.068995,0.059622,-47,-30,-30
531,1,-2,4.333333,3.113891,0.691008,9.184635,-3.448849,0.230558,0.478670,-4.190819,-0.033373,-0.019899,0.098608,0.051604,0.051163,-2,-10,-13
532,1,0,-1.151515,1.408000,-4.634516,17.866139,-4.307334,0.644280,2.083128,-6.789446,-0.058145,-0.061850,0.124211,0.000723,0.016367,9,2,2


Remove team ID and season -> cannot be scaled and not a metric. Dummy 

*** Also cut data after 2014 so the model isnt trained on test data***

In [25]:
# df_finalData = df_finalData[df_finalData<2014]
df_finalData.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
# df_finalData.head()

# Combine into test dataframe

In [26]:
df_finalDataTest = pd.concat((df_winlossTest, df_losswinTest))
results = df_finalDataTest['Result']
df_finalDataTest.drop(labels=['Result'], inplace=True, axis=1)
df_finalDataTest.insert(0, 'Result', results)
df_finalDataTest.shape

(402, 38)

In [27]:
df_finalDataTest.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
# df_finalData.head()

In [28]:
df_finalData.isnull().any()

Result       False
Seed         False
Pts          False
Pos          False
OffRtg       False
DefRtg       False
NetRtg       False
AstR         False
TOR          False
TSP          False
eFGP         False
FTAR         False
ORP          False
DRP          False
RP           False
MAS          False
SAG          False
POM          False
OppSeed      False
OppPts       False
OppPos       False
OppOffRtg    False
OppDefRtg    False
OppNetRtg    False
OppAstR      False
OppTOR       False
OppTSP       False
OppeFGP      False
OppFTAR      False
OppORP       False
OppDRP       False
OppRP        False
OppMAS       False
OppSAG       False
OppPOM       False
dtype: bool

In [29]:
df_finalData.to_csv('MarchMadnessFeatures.csv', index=False)

In [30]:
df_finalDataTest.head()

,Result,Seed,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,...,OppTOR,OppTSP,OppeFGP,OppFTAR,OppORP,OppDRP,OppRP,OppMAS,OppSAG,OppPOM
332,1,16,65.393939,65.800145,99.031266,101.542102,-12.428930,11.517922,9.785012,48.998897,...,12.709611,55.569454,0.501468,0.498555,0.304314,0.662497,0.478113,127,159,146
333,1,11,71.125000,63.668400,112.107238,107.163409,-5.271302,13.995073,13.039999,53.289440,...,14.090807,59.175634,0.534865,0.430636,0.341857,0.717383,0.532549,31,36,31
334,1,11,67.606061,61.341091,110.128215,95.059351,-4.063521,16.069185,13.542018,56.765912,...,11.711904,57.908993,0.538050,0.333966,0.273743,0.741725,0.499740,53,45,39
335,1,16,68.333333,63.844073,107.440591,106.231815,-9.445318,13.553533,13.067373,54.243134,...,11.974853,58.518232,0.536156,0.411573,0.288443,0.683750,0.505035,132,143,127
336,1,2,76.382353,63.633882,119.630663,93.106960,-0.529123,15.351748,11.766410,57.853189,...,11.834942,52.283856,0.495406,0.475098,0.338206,0.664771,0.506808,143,204,207


In [31]:
df_finalDataTest.to_csv('MarchMadnessTest.csv', index=False)

In [32]:
df_diffData.to_csv('MarchMadnessFeatureDifferences.csv', index=False)

In [33]:
df_Adv.to_csv('MarchMadnessAdvStats.csv', index=False)